# Import des librairies

In [8]:
import pandas as pd
import requests
import json
import urllib.parse
from bs4 import BeautifulSoup
import time
import random
import boto3
from io import StringIO
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

# Configuration AWS
AWS_REGION = 'eu-west-3'
BUCKET_NAME = 'projey-kayak'
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

# Initialiser le client S3
s3_client = boto3.client(
    's3',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

# Chargement du top 5 des villes

In [16]:
# Charger le TOP 5 des villes depuis S3
response = s3_client.get_object(
    Bucket=BUCKET_NAME,
    Key='processed/weather/top_5_cities.csv'
)
df_top_5 = pd.read_csv(StringIO(response['Body'].read().decode('utf-8')))

cities = df_top_5['city'].tolist()

print(f"✅ TOP 5 chargé : {cities}")

✅ TOP 5 chargé : ['Paris', 'Mont Saint Michel', 'Bayeux', 'Le Havre', 'Lille']


# Scraper les 5 villes et récupérer les infos sur les hotels

## Initialisation

In [9]:
#Initialiser un user-agent pour les requêtes HTTP
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Cache-Control": "max-age=0"
}

## Fonction

In [10]:
def extract_hotel_info(hotel):
    """
    Extrait les informations d'un hôtel.

    Retourne un dictionnaire avec le nom, l'url, la description et la note.
    """

    info = {
        "name": None,
        "url": None,
        "description": None,
        "rating": None
    }

    #Nom de l'hôtel
    name = hotel.find('div', {"data-testid": "title"})
    if name:
        info["name"] = name.get_text(strip=True)

    #URL de l'hôtel
    link = hotel.find('a', {"data-testid": "title-link"})
    if link and link.get('href'):
        info['url'] = link['href']

    #Note de l'hôtel
    rating = hotel.find('div', {"data-testid": "review-score"})
    if rating:
        rating_div = rating.find("div", class_='dff2e52086')
        if not rating_div:
            #Autre possibilité de classe
            rating_div = rating.find('div', class_='f63b14ab7a dff2e52086')
        if rating_div:
            info['rating'] = rating_div.get_text(strip=True)

    #Description de l'hôtel
    #La description est dans une div avec la classe 'fff1944c52'
    potential_desc = hotel.find_all('div', class_='fff1944c52')
    for desc in potential_desc:
        text = desc.get_text(strip=True)
        #La description est un texte long qui ne contient pas "Indiquer sur la carte"
        if len(text) > 50 and "Indiquer sur la carte" not in text:
            info['description'] = text
            break

    return info

## Extraction

In [ ]:
# Scraping
all_hotels = []
 
for city in cities:
    encoded_city = urllib.parse.quote_plus(city + ", France")
    url = f"https://www.booking.com/searchresults.html?ss={encoded_city}&order=bayesian_review_score&nflt=ht_id%3D204"
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    hotels = soup.find_all('div', {"data-testid": "property-card"})
    
    for hotel in hotels[:5]:
        hotel_info = extract_hotel_info(hotel)
        hotel_info['city'] = city
        all_hotels.append(hotel_info)
    
    print(f"✅ {city}: {len(hotels[:5])} hôtels extraits")
    time.sleep(random.uniform(10, 15))

In [14]:
# Créer le DataFrame avec les hôtels
df_hotels = pd.DataFrame(all_hotels)

# Réorganiser les colonnes
colonnes = ["city", "name", "url", "description", "rating"]
df_hotels = df_hotels[colonnes]

print(f"✅ DataFrame créé : {len(df_hotels)} hôtels")
print(df_hotels.head())

✅ DataFrame créé : 25 hôtels
     city                   name  \
0  Bayeux       Villa Lara Hotel   
1  Bayeux      Domaine de Bayeux   
2  Bayeux  La Maison de Mathilde   
3  Bayeux         Belle Normandy   
4  Bayeux       Hôtel d'Argouges   

                                                 url  \
0  https://www.booking.com/hotel/fr/villa-lara-ha...   
1  https://www.booking.com/hotel/fr/domaine-de-ba...   
2  https://www.booking.com/hotel/fr/la-maison-de-...   
3  https://www.booking.com/hotel/fr/clos-des-cord...   
4  https://www.booking.com/hotel/fr/d-argouges.fr...   

                                         description rating  
0  La Villa Lara Hotel est un petit boutique hôte...    9,6  
1  Le Domaine de Bayeux occupe une maison du XVII...    9,3  
2  La Maison de Mathilde vous accueille à Bayeux,...    8,9  
3  L’établissement Belle Normandy se trouve à Bay...    8,8  
4  L'Hôtel d'Argouges est un établissement du XVI...    8,7  


# Export des données

In [15]:
# Uploader les données brutes des hôtels sur S3
csv_buffer = StringIO()
df_hotels.to_csv(csv_buffer, index=False)

s3_client.put_object(
    Bucket=BUCKET_NAME,
    Key='raw/hotels/booking_hotels_raw.csv',
    Body=csv_buffer.getvalue()
)

print("✅ df_hotels uploadé sur S3 (raw/hotels/booking_hotels_raw.csv)")

✅ df_hotels uploadé sur S3 (raw/hotels/booking_hotels_raw.csv)
